In [ ]:
import itertools
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from math import sqrt
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Input, TimeDistributed, RepeatVector, concatenate, Layer
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import Callback
from datetime import datetime
import time
import pandas as pd
from datetime import datetime

import ast

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if tf.test.gpu_device_name():
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [3]:
#reset the training and test by time steps
def to_supervised(data, space, ts_xandy):
    for i in range((len(data))):
        cases=data[i,:,:]
        list_case=list()
        in_start=0
        for j in range((len(cases))):
            in_end=in_start+ts_xandy
            if in_end<=len(cases):
                list_case.append(cases[in_start:in_end,:])
            in_start+=space
        list_case=np.array(list_case)
        if i == 0:
            final=list_case
        else:
            final=np.vstack((final,list_case))
    return final


#process the data
def process_data (ph, space, df_all_o, list_colmns, number_location_static):
    ti=5 #time interval
    begining_time=65 #from 60 minutes before incident happens

    number_of_ti=4*60/ti#evevry case how many time steps
    start=int((65-begining_time)/ti)
    end=int(number_of_ti) #the mamximun time step that this model can predict
    ts_xandy=int((begining_time+ph)/ti)
    split_y=int(begining_time/ti)


    df_all=df_all_o.loc[:,list_colmns]
    df_all=df_all.reset_index(drop=True)

    #make sure no missing data
    for i in df_all.columns:
        se=df_all[i].copy()
        index_outliers=se[abs(se-se.mean())>4*se.std()].index.to_list()
        se.iloc[index_outliers]=np.nan
        df_all[i]=se
    df_all=df_all.fillna(method="backfill")
    nan_df=pd.isna(df_all).sum()

    #scale x, standerdization
    df_nospeed=df_all.iloc[:,1:]
    df_nospeed=(df_nospeed-df_nospeed.mean())/df_nospeed.std()
    df_model=df_nospeed.copy()
    speed=df_all['speed']
    df_model['speed']=speed


    df_model=df_model.loc[:,list_colmns]

    #sequence of speed y
    or_speed=0

    #split training, validation, test
    values=df_model.values
    values = np.array(np.split(values, len(values)/number_of_ti))
    values=values[:,start:end,:]

    #split training and test
    np.random.seed(688)
    indices = np.random.permutation(values.shape[0])
    num_train=int(values.shape[0]*0.7)
    training_idx, test_idx = indices[:num_train], indices[num_train:]

    training, test = values[training_idx,:], values[test_idx,:]

    training_su=to_supervised(training, space, ts_xandy)
    test_su=to_supervised(test, space, ts_xandy)

    #split x,y

    training_x, training_y=training_su[:,:split_y,:], training_su[:,split_y:,or_speed]
    test_x,test_y=test_su[:,:split_y,:], test_su[:,split_y:,or_speed]


    sequential_input_data=training_x[:,:,:number_location_static]
    static_input_data=training_x[:,0,number_location_static:]
    output_data=training_y

    test_sequ_input=test_x[:,:,:number_location_static]
    test_sta_input=test_x[:,0,number_location_static:]
    test_output=test_y


    static_input_dim=static_input_data.shape[1]

    n_timesteps=sequential_input_data.shape[1]
    seq_input_dim=sequential_input_data.shape[2]

    output_dim=output_data.shape[1]

    return (sequential_input_data, static_input_data, output_data,
            test_sequ_input, test_sta_input, test_output,
            static_input_dim, n_timesteps, seq_input_dim, output_dim,
            test,ts_xandy, split_y, test_idx, training, training_idx)




In [ ]:
#new test and training accuracy

def performance (model, test, ts_xandy, split_y, test_idx):
  performance_each_test=pd.DataFrame(columns=['number','MAPE','RMSE','MAE'])
  for i in range(len(test)):
      #make sure i's shape is (1,48,7)
      test_i=test[i,:,:]
      test_i=test_i.reshape(1,test_i.shape[0],test_i.shape[1])

      test_su=to_supervised(test_i, space, ts_xandy)

      test_x,test_y=test_su[:,:split_y,:], test_su[:,split_y:,0]

      test_sequ_input=test_x[:,:,:number_location_static]
      test_sta_input=test_x[:,0,number_location_static:]
      test_output=test_y

      ythat=model.predict([test_sta_input, test_sequ_input], verbose=0, )
      ythat=ythat.reshape(ythat.shape[0],ythat.shape[1])

      mape_test=mean_absolute_percentage_error(test_y, ythat)
      rmse_test = sqrt(mean_squared_error(test_y, ythat))
      #caculate MAE
      mae_test=np.mean(np.abs(test_y-ythat))


      performance_each_test.loc[i]=[i,mape_test,rmse_test,mae_test]

  #add test index in performance_each_test
  performance_each_test['test_index']=test_idx
  return performance_each_test


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer


class BahdanauAttentionLayer(Layer):
    def __init__(self, attention_dim, **kwargs):

        self.attention_dim = attention_dim
        super(BahdanauAttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W1 = self.add_weight(
            name='attention_weight1',
            shape=(input_shape[-1], self.attention_dim),
            initializer='random_normal',
            trainable=True
        )
        self.W2 = self.add_weight(
            name='attention_weight2',
            shape=(input_shape[-1], self.attention_dim),
            initializer='random_normal',
            trainable=True
        )
        self.V = self.add_weight(
            name='attention_value',
            shape=(self.attention_dim, 1),
            initializer='random_normal',
            trainable=True
        )
        super(BahdanauAttentionLayer, self).build(input_shape)

    def call(self, x):
        query_with_time_axis = tf.expand_dims(x[:, -1, :], 1)  # Last hidden state as the query

        # Custom scoring function with adjusted dimensions
        score = tf.nn.tanh(tf.matmul(query_with_time_axis, self.W1) + tf.matmul(x, self.W2))
        alignment_scores = tf.matmul(score, self.V)

        # Softmax to get the attention weights
        attention_weights = tf.nn.softmax(alignment_scores, axis=1)

        # Context vector
        context_vector = attention_weights * x
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector

    def get_config(self):
        config = super(BahdanauAttentionLayer, self).get_config()
        config['attention_dim'] = self.attention_dim
        return config


#set the model structure
def define_model(static_input_dim, n_timesteps, seq_input_dim, output_dim,
                 encodedecode_neurons, reg_technique, optimizer, activation,
                 learning_rate, dropout_rate, attention_type):
    # Regularization
    if reg_technique is not None:
        if reg_technique == 'l1(0.01)':
            reg = regularizers.l1(l=0.01)
    else:
        reg = None

    # Static branch
    inputA = Input(shape=(static_input_dim,))
    layer1 = Dense(encodedecode_neurons*4, activation=activation, kernel_regularizer=reg)(inputA)
    layer2 = Dense(encodedecode_neurons*2, activation=activation, kernel_regularizer=reg)(layer1)


    xstatic = Model(inputs=inputA, outputs=layer2)

    # Sequence branch
    input_layer = Input(shape=(n_timesteps, seq_input_dim))
    x = LSTM(encodedecode_neurons*8, activation=activation, return_sequences=True, kernel_regularizer=reg)(input_layer)
    x = LSTM(encodedecode_neurons*4, activation=activation, return_sequences=True, kernel_regularizer=reg)(x)
    x = LSTM(encodedecode_neurons*2, activation=activation, return_sequences=True, kernel_regularizer=reg)(x)
    x = LSTM(encodedecode_neurons*2, activation=activation, return_sequences=True, kernel_regularizer=reg)(x)
    x = LSTM(encodedecode_neurons, activation=activation, return_sequences=True, kernel_regularizer=reg)(x)
    x = LSTM(encodedecode_neurons, activation=activation, return_sequences=True, kernel_regularizer=reg)(x)

    # Attention layer
    if attention_type == 'bahdanau':
        attention = BahdanauAttentionLayer(attention_dim=64)(x)  # Assuming BahdanauAttentionLayer is defined elsewhere


    x = Model(inputs=input_layer, outputs=attention)

    # Combining the outputs
    combined = concatenate([xstatic.output, x.output])

    # Repeat vector layer
    combined = RepeatVector(output_dim)(combined)
    combined = LSTM(encodedecode_neurons, activation=activation, return_sequences=True)(combined)



    # Time distributed dense layers
    z = TimeDistributed(Dense(encodedecode_neurons, activation=activation))(combined)
    z = TimeDistributed(Dense(1))(z)

    # Final model
    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif optimizer == 'adagrad':
        opt = tf.keras.optimizers.Adagrad(learning_rate=learning_rate)
    elif optimizer == 'adadelta':
        opt = tf.keras.optimizers.Adadelta(learning_rate=learning_rate)
    elif optimizer == 'adamax':
        opt = tf.keras.optimizers.Adamax(learning_rate=learning_rate)
    # ... Add other optimizers similarly
    elif optimizer == 'nadam':
        opt = tf.keras.optimizers.Nadam(learning_rate=learning_rate)

    model = Model(inputs=[xstatic.input, x.input], outputs=z)
    model.compile(optimizer=opt, loss='mse', metrics=[tf.keras.metrics.MeanAbsolutePercentageError()])

    return model

In [ ]:
# Placeholder for results
results = []

file_path='data.xlsx'
df_all_o = pd.read_excel(file_path)

list_colmns=['speed','flow','std_flow','std_speed',
             'distance','NOL','severity',]
list_per=['MAPE','RMSE','MAE']
number_location_static=4
#set epochs
epochs=30
#set the params
params=(2,None, 'adam', 'tanh', 16, 0.001, 0, 'bahdanau',15)
space, reg_technique, optimizer, activation, encodedecode_neurons, learning_rate, dropout_rate, attention_type,ph = params

(sequential_input_data, static_input_data,
    output_data,test_sequ_input, test_sta_input,
    test_output,static_input_dim, n_timesteps,
    seq_input_dim, output_dim,test,ts_xandy,
    split_y, test_idx, training,training_idx)= process_data (ph, space,df_all_o, list_colmns, number_location_static)

# Define and compile your model
model = define_model(static_input_dim, n_timesteps, seq_input_dim, output_dim,
                        encodedecode_neurons, reg_technique, optimizer, activation,
                        learning_rate, dropout_rate, attention_type)
start_time = time.time()
# Train the model
history = model.fit([static_input_data, sequential_input_data], output_data, 
                    epochs=epochs, validation_split=0.25)
# End the timer
end_time = time.time()
# Calculate the elapsed time
elapsed_time = (end_time - start_time)/epochs


perf_test_df=performance (model, test, ts_xandy, split_y, test_idx)
this_test=perf_test_df[list_per].mean()

print(this_test)
print(elapsed_time)